In [1]:
import requests
import pandas as pd
import re
import numpy as np

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'}
url_pattern = re.compile(r"""<a href="/view_video.php\?viewkey=(.*?)" """)

In [3]:
def get_url(url):
  response = requests.get(url, headers=headers)
  content = response.text
  return content

In [ ]:
data = []
last_page = 440
page_number = 1

while page_number <= last_page:
  try:
    url = f'https://pl.pornhub.com/video?p=professional&page={page_number}'
    content = get_url(url)
    advs = re.findall(url_pattern, content)
    for x in advs:
      data.append(x)
  except:
    continue
  page_number +=1

In [ ]:
data = []
last_page = 440
page_number = 1

while page_number <= last_page:
  try:
    url = f'https://pl.pornhub.com/video?p=homemade&page={page_number}'
    content = get_url(url)
    advs = re.findall(url_pattern, content)
    for x in advs:
      data.append(x)
  except:
    continue
  page_number +=1

In [ ]:
data = set(data)

In [ ]:
len(data)

In [ ]:
url_pattern = re.compile(r"""<a href="/view_video.php\?viewkey=(.*?)" """)
time_pattern = re.compile(r"""<div id="js-shareData" class="tab-menu-wrapper-table" data-duration="(.*?)" data-share-url""")
title_pattern = re.compile(r"""<title>(.*?)(?: - Pornhub\.com| &#124; Pornhub)<\/title>""")
cat_pattern = re.compile(r"""'categories_in_video' : '(.*?)'""")
views_pattern = re.compile(r"""<div class="views"><span class="count">(.*?)</span> Views</div>""")
tag_pattern = re.compile(r'<a data-event="video_underplayer" data-label="tag" class="gtm-event-video-underplayer item" href="/video/search\?search=(\w.*?)"')

In [ ]:
def get_film(film_url):
  url = f'https://pornhub.com/view_video.php?viewkey={film_url}'
  response = requests.get(url, headers=headers)
  return response.text

In [ ]:
def get_time(film_content):
  time = re.search(time_pattern, film_content)
  return time.group(1)

In [ ]:
def get_title(film_content):
  title = re.search(title_pattern, film_content)
  return title.group(1)

In [ ]:
def get_cat(film_content):
  cat = re.search(cat_pattern, film_content)
  return cat.group(1)

In [ ]:
def get_tags(film_content):
  tags = re.findall(tag_pattern, film_content)
  return tags

In [ ]:
def val_to_bool(value):
    if value[-1] == 'M':
        return 1
    elif value[-1] == 'K':
        return 0
    else:
        return np.NaN

In [ ]:
def get_views(film_content):
  views = re.search(views_pattern, film_content)
  return views.group(1)

In [ ]:
list_of_data = []
views_list = []
time_list = []
title_list = []

for link in data:
  text = get_film(link)
  try:
    cat = get_cat(text)
    cat_list = list(cat.split(","))
    views = val_to_bool(get_views(text))
    time = get_time(text)
    title = get_title(text)
    tags = get_tags(text)

  except:
    continue
  views_list.append(views)
  time_list.append(time)
  title_list.append(title)
  list_of_data.append(cat_list + tags)

In [ ]:
unique_categories = set(category for category_list in list_of_data for category in category_list)

In [ ]:
len(unique_categories)

In [ ]:
list_1 = list_of_data[:4799]
list_2 = list_of_data[4799:9598]
list_3 = list_of_data[9598:14397]
list_4 = list_of_data[14397:19348]

In [ ]:
#Podział na 4 jest stąd że trwa to w nieskończoność, niewiadomo dlaczego

def create_category_presence_dict(category_lists, unique_categories):
    category_dict = {
        category: [True if category in category_list else False for category_list in category_lists]
        for category in unique_categories
    }
    return category_dict


l1 = create_category_presence_dict(list_1, unique_categories)
l2 = create_category_presence_dict(list_2, unique_categories)
l3 = create_category_presence_dict(list_3, unique_categories)
l4 = create_category_presence_dict(list_4, unique_categories)

In [ ]:
l11 = pd.DataFrame(l1, dtype='bool')
l12 = pd.DataFrame(l2, dtype='bool')
l13 = pd.DataFrame(l3, dtype='bool')
l14 = pd.DataFrame(l4, dtype='bool')

y = pd.DataFrame(views_list, dtype='bool')
title = pd.DataFrame(title_list)
time = pd.DataFrame(time_list)

In [ ]:
y.rename(columns={0: 'kolumna_celi',}, inplace=True)
time.rename(columns={0: 'czas',}, inplace=True)
title.rename(columns={0: 'tytuł',}, inplace=True)

In [ ]:
X = pd.concat([l11, l12, l13, l14], ignore_index=True) 

In [ ]:
X.to_csv('X.csv', index=False)
y.to_csv('y.csv', index=False)
time.to_csv('time.csv', index=False)
title.to_csv('title.csv', index=False)